In [ ]:
# todolist.py

import os
import pygame

pygame.init()
SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 600

WHITE = (255, 255, 255)
BLACK = (0,0,0)
BLUE = (0, 0, 255)
PEACH = (241, 95, 95)
FILE_NAME = 'todolist.txt'

screen = pygame.display.set_mode( (SCREEN_WIDTH, SCREEN_HEIGHT) )
pygame.display.set_caption("To do list")
clock = pygame.time.Clock()
font = pygame.font.SysFont("malgungothic", 48)

class TaskManager:
    def __init__(self):
        self.tasks = self.load_tasks()

    def load_tasks(self):
        if not os.path.exists(FILE_NAME): # 현재 디렉터리에 파일 이름이 있는지 검사
            return {}
        with open(FILE_NAME, 'r') as file:
            tasks = {}
            for line in file:
                task_id, task_desc = line.strip().split(':', 1)
                tasks[int(task_id)] = task_desc
            return tasks

    def save_tasks(self):
        with open(FILE_NAME, 'w') as file:
            for task_id, task_desc in self.tasks.items():
                file.write(f"{task_id}:{task_desc}\n")

    def add_task(self, task_desc):
        task_id = max(self.tasks.keys(), default = 0) + 1
        self.tasks[task_id] = task_desc
        self.save_tasks()

    def delete_task(self, task_id):
        if task_id in self.tasks:
            del self.tasks[task_id]
            self.save_tasks()

    def edit_task(self, task_id, new_desc):
        if task_id in self.tasks:
            self.tasks[task_id] = new_desc
            self.save_tasks()

    def get_task(self, task_id):
        return self.tasks.get(task_id)

    def get_tasks(self):
        return self.tasks

class TaskInputBox(pygame.Rect):
    def __init__(self, x, y, w, h):
        super().__init__(x, y, w, h)
        self.text = ''
        self.active = False

    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            if self.collidepoint(event.pos):
                self.active = True
            else:
                self.active = False

        if event.type == pygame.KEYDOWN and self.active:
            if event.key == pygame.K_RETURN:
                return self.text
            elif event.key == pygame.K_BACKSPACE:
                self.text = self.text[:-1]
            else:
                self.text += event.unicode
        return None

    def draw(self):
        pygame.draw.rect(screen, PEACH, self, 2)
        draw_text(self.text, position=(self.x + 5, self.y + 5))

    def set_text(self, text):
        self.text = text

def draw_text(text, position=(50, 10), color=BLACK):
    text_obj = font.render(text, True, color)
    screen.blit(text_obj, position)

def draw_tasks(tasks):
    nxt_y = 100
    for task_id, task_desc in tasks.items():
        draw_text(f"{task_id}:{task_desc}", position=(50, nxt_y))
        nxt_y += 50

if __name__ == "__main__":
    tm = TaskManager()
    input_box = TaskInputBox(50, 100, 700, 70)
    mode = 'view'
    edit_input_box = TaskInputBox(50, 100, 700, 70)
    edit_task_id = None

    while True:
        screen.fill(WHITE)


        if mode == 'view':
            draw_text("Press 'a' to add, 'd' to delete, 'e' to edit")
            draw_tasks(tm.get_tasks())  # draw_tasks(tm.tasks)
        elif mode == 'add':
            draw_text("Enter new task : ")
            input_box.draw()
        elif mode == 'delete':
            draw_text("Enter ID to delete : ")
            input_box.draw()
        elif mode == 'edit':
            if edit_task_id is None:
                draw_text("Enter ID to edit : ")
                input_box.draw()
            else:
                draw_text(f"Editing task {edit_task_id}. Enter new description")
                edit_input_box.draw()

        # event 감지
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()

            if mode == 'view':
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_a:
                        mode = 'add'
                        input_box.set_text('')  # input_box.text = ''
                    elif event.key == pygame.K_d:
                        mode = 'delete'
                        input_box.set_text('')
                    elif event.key == pygame.K_e:
                        mode = 'edit'
                        input_box.set_text('')
            elif mode == 'add':
                task_desc = input_box.handle_event(event)
                if task_desc is not None:
                    tm.add_task(task_desc)
                    mode = 'view'
            elif mode == 'delete':
                task_id = input_box.handle_event(event)
                if task_id is not None:
                    try:
                        task_id = int(task_id)
                        tm.delete_task(task_id)
                    except ValueError:
                        pass
                    mode = 'view'
            elif mode == 'edit':
                if edit_task_id is None:
                    task_id = input_box.handle_event(event)
                    print(task_id)
                    if task_id is not None:
                        try:
                            task_id = int(task_id)
                            task_desc = tm.get_task(task_id)
                            if task_desc is not None:
                                edit_input_box.set_text(task_desc)
                                edit_task_id = task_id
                        except ValueError:
                            pass
                else:
                    new_desc = edit_input_box.handle_event(event)
                    if new_desc is not None:
                        tm.edit_task(edit_task_id, new_desc)
                        mode = 'view'
                        edit_task_id = None


        pygame.display.flip()
        clock.tick(30)  # 초 당 FRAME 설정

pygame 2.6.0 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyboardInterrupt: 

In [7]:
# web.py

import http.server

class SimpleHandler(http.server.BaseHTTPRequestHandler):
  # HTTP로 네트워크 상에서 통신이 이루어짐
  # 그 통신의 종류가 크게 두 가지 : GET, POST
  # 그 중에서 제일 많이 쓰이는 통신 종류가 GET
  def do_GET(self):
    self.send_response(200)
    self.send_header('Content-type', 'text/html')
    self.end_headers()
    self.wfile.write(b'Hello, world!')

# IP 주소 : 네트워크에서 접근할 수 있는 숫자로 이뤄진 주소값
# 도메인 : IP주소를 문자열로 등록해서 똑같이 접근 가능
server = http.server.HTTPServer(('localhost', 8080), SimpleHandler)
# localhost 도메인으로, 8080 포트에 가상의 Web Server 생성
# 포트 번호 : IP 주소가 집 주소라면, 포트 번호는 출입구 번호
print("Starting server at http://localhost:8080")
server.serve_forever()    # 영속적으로 Web Server 가동

OSError: [Errno 98] Address already in use

In [ ]:
# example.py

import requests   # command prompt => pip install requests
from bs4 import BeautifulSoup as bs   # pip install BeautifulSoup4
# requests 모듈 : url로 웹페이지의 내용을 가져옴
# BeautifulSoup : 웹페이지 내용을 좀 더 정갈하게 다듬어주는 모듈

# 예시 url : 'https://sports.news.naver.com/kbaseball/record/index?category=kbo'
url = 'https://sports.news.naver.com/kbaseball/record/index?category=kbo'

response = requests.get(url)
print(response.status_code)   # 상태 코드 출력 / 이는 200 이 출력될 때만 성공이다.

html = response.text
print(html)

200
<!DOCTYPE html>
<html lang="ko">

<head>
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
    <meta name="viewport" content="width=1080px,maximum-scale=2.0,minimum-scale=0.4,user-scalable=yes">
    <meta property="og:image"       content="https://imgsports.pstatic.net/images/mobile/common/NAVERsports.png"/>
    <meta property="og:title"       content="네이버 스포츠"/>
    <meta property="og:description" content="스포츠의 시작과 끝!"/>
    <meta property="og:url"
          content="sports.news.naver.com/kbaseball/record/index?category=kbo&tab=team"/>
    
        
        
        
            <link rel='stylesheet' type='text/css' href='https://ssl.pstatic.net/static.sports/resources/pc/272727/66/278/665600/css/common_loose_new.css'>

            <link rel='stylesheet' type='text/css' href='https://ssl.pstatic.net/static.sports/resources/pc/272727/66/278/665600/css/baseball_gamecenter.css'>

        
        

     